In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from proyeccion_rdr.produccion.a05_unidades_de_apoyo import guardar_dict_en_excel

ANIOS_POBLACION = [f"{i}" for i in [2025, 2030, 2040]]

2025-07-03 22:11:34.495 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr


In [3]:
# Lee la poblacion FONASA
ruta_poblaciones = "../data/interim/0.0_poblaciones_ine_y_fonasa_a_utilizar.xlsx"
poblaciones_fonasa = pd.read_excel(ruta_poblaciones, sheet_name="poblacion_FONASA")

# Filtra solamente la poblacion del SSMN y entre 0 y 15 anios (Poblacion que efectivamente
# atendieron). Elimina la columna del 2023, ya que esta mal calculada
poblaciones_fonasa_ssmn = (
    poblaciones_fonasa.query("Estrato == 'SSMN' and `Edad Incidencia` == 'entre_0_y_15'")
    .set_index(["Edad Incidencia", "Estrato"])
    .drop(columns=["2023"])
)
display(poblaciones_fonasa_ssmn)

,,2018,2019,2020,2021,2022
Edad Incidencia,Estrato,,,,,
entre_0_y_15,SSMN,148276,155320,153830,151308,153566


In [4]:
# Lee las consultas de urgencias REM
produccion_urgencias_rem = pd.DataFrame(
    [
        [59, 70, 82, 65, 87, 69, 91, 448, 176, 169],
        [1455, 1292, 2133, 2635, 3096, 1499, 5235, 13613, 12473, 12981],
        [33159, 28148, 25438, 22907, 27428, 13795, 20488, 29809, 28980, 29070],
        [66592, 64954, 63160, 62138, 54425, 21814, 22329, 36763, 37584, 38696],
        [4841, 4781, 0, 0, 1, 0, 0, 0, 0, 0],
        [3764, 3302, 5514, 11914, 8251, 2476, 1484, 933, 1237, 1250],
        [109870, 102547, 96327, 99659, 93288, 39653, 49627, 81566, 80450, 82166],
    ],
    index=["C1", "C2", "C3", "C4", "SIN CATEGORIZACION", "C5", "Total"],
    columns=["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"],
)
total_urgencias_rem = produccion_urgencias_rem.loc["Total"]

# Obtiene el porcentaje de cada categoria respecto al total
porcentaje_categorizacion_rem = produccion_urgencias_rem / produccion_urgencias_rem.iloc[-1]

In [5]:
# Obtiene la proporcion de urgencias y poblacion
proporcion_urgencias_y_poblacion = total_urgencias_rem / poblaciones_fonasa_ssmn

In [6]:
# Selecciona la cantidad de urgenicas por personas.
# De 15 a 18 fue criterio de experto - Dr Tapia
anio_maximo = proporcion_urgencias_y_poblacion.idxmax(axis=1).iloc[0]
texto_maximo = f"Maximo de urgencias por persona histórico SSMN: {anio_maximo}"
factor_urgencias_0_a_14 = proporcion_urgencias_y_poblacion[anio_maximo].iloc[0]
factor_urgencias_15_a_18 = 1.3

# Extrae los factores de categorizacion para proyectar
factores_categorizacion = porcentaje_categorizacion_rem[anio_maximo].to_frame()
factores_categorizacion["criterio_eleccion"] = texto_maximo
factores_categorizacion = factores_categorizacion.rename(
    columns={anio_maximo: "porcentaje_con_consultas_urg"}
)

# Consolida los factores de urgencias
resumen_factores_elegidos = pd.DataFrame(
    index=["recien_nacidos_vivos", "entre_1_y_14", "entre_15_y_18"],
    columns=["porcentaje_con_consultas_urg", "consultas_por_paciente", "criterio_eleccion"],
    data=[
        [factor_urgencias_0_a_14, 1, texto_maximo],
        [factor_urgencias_0_a_14, 1, texto_maximo],
        [1, factor_urgencias_15_a_18, "Criterio Experto - Dr Tapia"],
    ],
)

# Concatena factores de categorizacion a las variables
resumen_factores_elegidos = pd.concat([resumen_factores_elegidos, factores_categorizacion])

In [7]:
# Lee la poblacion SSMN extrapolada
estratos_interes_0_a_18 = ["recien_nacidos_vivos", "entre_1_y_14", "entre_15_y_18"]

# Extrae solamente los estratos de interes
poblacion_fonasa_extrapolada_ssmn = (
    pd.read_excel(ruta_poblaciones, sheet_name="pobl_fonasa_extrapol_censo_2024")
    .query("Estrato == 'SSMN'")
    .set_index("Edad Incidencia")
    .drop(columns="Estrato")
    .query("`Edad Incidencia`.isin(@estratos_interes_0_a_18)")
)


In [8]:
# Multiplica la cantidad de poblacion por el porcentaje que requiere urgencia
pacientes_con_urgencia = poblacion_fonasa_extrapolada_ssmn.mul(
    resumen_factores_elegidos["porcentaje_con_consultas_urg"], axis=0
).dropna()

# Obtiene la cantidad de consultas de urgencia
consultas_urgencia = pacientes_con_urgencia.mul(
    resumen_factores_elegidos["consultas_por_paciente"], axis=0
).dropna()

# Agrega el total de consultas
consultas_urgencia.loc["Total"] = consultas_urgencia.sum()

# Calcula por porcentaje de categorizacion
consultas_categorizacion = pd.DataFrame(
    index=factores_categorizacion.index, data=[consultas_urgencia.loc["Total"]]
)

# Multiplica y calcula las consultas de categorizacion
consultas_categorizacion = (
    consultas_categorizacion.mul(resumen_factores_elegidos["porcentaje_con_consultas_urg"], axis=0)
    .dropna()
    .drop(index="Total")
)

# Resumen final
resumen_consultas = pd.concat([consultas_urgencia, consultas_categorizacion])

In [9]:
print(f"Total consultas urgencia proyectadas 2040: {resumen_consultas.loc['Total', '2040']:.2f}")

Total consultas urgencia proyectadas 2040: 179397.94


In [10]:
resumen_minsal = pd.DataFrame(
    {
        "poblacion_fonasa_2040": poblacion_fonasa_extrapolada_ssmn["2040"],
        "porcentaje_paciente_con_consulta_urgencia": resumen_factores_elegidos[
            "porcentaje_con_consultas_urg"
        ],
        "pacientes_urgencia_2040": pacientes_con_urgencia["2040"],
        "cantidad_consultas_urgencia_por_paciente": resumen_factores_elegidos[
            "consultas_por_paciente"
        ],
        "consultas_urgencia_2040": consultas_urgencia["2040"],
    }
).dropna()

## Resumen MINSAL

In [11]:
a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "poblaciones_fonasa_ssmn": poblaciones_fonasa_ssmn,
    "produccion_urgencias_rem": produccion_urgencias_rem,
    "porcentaje_categorizacion_rem": porcentaje_categorizacion_rem,
    "porcentaje_urgencias_pobl_ssmn": proporcion_urgencias_y_poblacion,
    "factores_para_proyectar": resumen_factores_elegidos,
    "poblacion_ssmn_extrapol": poblacion_fonasa_extrapolada_ssmn,
    "pacientes_con_urgencia": pacientes_con_urgencia,
    "proyeccion_consultas": resumen_consultas
}

In [12]:
ruta_archivo = "../data/interim/4.0_estimacion_consultas_urgencia.xlsx"
with pd.ExcelWriter(ruta_archivo) as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)